In [1]:
import numpy as np
import pandas as pd
import re
from noahmptbl_io import float_nsf,write_noahmptbl,read_noahmp_table
import os

In [4]:
sheet_url = "https://docs.google.com/spreadsheets/d/1odhiHtJRyWWKbeSGTuBwa-RiHB-wKV_kXFLa5tfwB0M/export?format=csv"
df = pd.read_csv(sheet_url,index_col=[0])

In [5]:
# select parameters that are available in UFS-land-driver
df_sel = df[df['Avail in UFS-land-driver']=='Yes']

In [6]:
df_sel

,Parameter description,Section,Exist in USGS parameter,"Value dependence (global, soil, veg)",Category,Continous or Integer,Min,Max,Default (MODIS),Standard or Hardcoded,Avail in UFS-land-driver,Source of default value,Parameter Selection Basis,Source for parameter range
Parameter name,,,,,,,,,,,,,,
nroot,number of soil layers from top down reached by...,noahmp_modis_parameters,Yes,veg,SoilP,Integer,0,4,"4, 4, 4, 4, 4, 3, ...",Standard,Yes,noahmptable.tbl,Figures 4&5 in Cuntz et al. 2016,NaN
hvt,Canopy top [m],noahmp_modis_parameters,Yes,veg,VegS,Continous,x0.5,x2,"20.0, 20.0, 18.0, 16.0, 16.0, 1.10, ...",Standard,Yes,noahmptable.tbl,Figures 4&5 in Cuntz et al. 2016,NaN
hvb,Canopy bottom [m],noahmp_modis_parameters,Yes,veg,VegS,Continous,x0.5,x2,"8.50, 8.00, 7.00, 11.5, 10.0, 0.10, ...",Standard,Yes,noahmptable.tbl,Figures 4&5 in Cuntz et al. 2016,NaN
rc,Crown radius [m],noahmp_modis_parameters,Yes,veg,VegS,Continous,x0.5,x2,"1.20, 3.60, 1.20, 1.40, 1.40, 0.12, ...",Standard,Yes,noahmptable.tbl,Figures 4&5 in Cuntz et al. 2016,NaN
cwpvt,Canopy wind parameter for canopy wind profile ...,noahmp_modis_parameters,Yes,veg,VegS,Continous,x0.5,x2,"0.09, 0.335, 0.09, 0.335, 0.145, 0.5, ...",Standard,Yes,noahmptable.tbl,Figures 4&5 in Cuntz et al. 2016,NaN
kc25,CO2 Michaelis-Menten constant [Pa],noahmp_modis_parameters,Yes,veg,Phys,Continous,x0.5,x2,"30.0, 30.0, 30.0, 30.0, 30.0, 30.0, ...",Standard,Yes,noahmptable.tbl,Figures 4&5 in Cuntz et al. 2016,NaN
akc,Q10 for KC25; relative changes to KC25 for 10o...,noahmp_modis_parameters,Yes,veg,Phys,Continous,x0.5,x2,"2.1, 2.1, 2.1, 2.1, 2.1, 2.1, ...",Standard,Yes,noahmptable.tbl,Figures 4&5 in Cuntz et al. 2016,NaN
ko25,O2 Michaelis-Menten constant [Pa],noahmp_modis_parameters,Yes,veg,Phys,Continous,x0.5,x2,"3.e4, 3.e4, 3.e4, 3.e4, 3.e4, 3.e4, ...",Standard,Yes,noahmptable.tbl,Figures 4&5 in Cuntz et al. 2016,NaN
ako,Q10 for KO25; relative changes to KO25 for 10o...,noahmp_modis_parameters,Yes,veg,Phys,Continous,x0.5,x2,"1.2, 1.2, 1.2, 1.2, 1.2, 1.2, ...",Standard,Yes,noahmptable.tbl,Figures 4&5 in Cuntz et al. 2016,NaN


In [7]:
# parameter lists
param_list = df_sel.index.values

oaat_exp_id = 0
oaat_exp_id_list = []
oaat_exp_param_list = []
oaat_exp_param_value_list = []

# number of interval 5
num_exp = 6

for param in param_list:
    value_type = df_sel['Continous or Integer'].loc[param]
    value_max = df_sel['Max'].loc[param]
    value_min = df_sel['Min'].loc[param]
    if value_type == "Integer":
        print(param, value_type)
        value_max = int(value_max)
        value_min = int(value_min)
        print(value_max,value_min)
        for value in range(value_min, value_max+1):
            oaat_exp_id+=1
            oaat_exp_id_list.append("oaat_%i"%oaat_exp_id)
            oaat_exp_param_list.append(param)
            oaat_exp_param_value_list.append(value)
    else:
        print(param, value_type)
        if value_max[0] == "x":
            value_min = np.float32(value_min[1:])
            value_max = np.float32(value_max[1:])
            step = (value_max-value_min)/(num_exp-1)
            for value in np.arange(value_min,value_max+1e-4,step):
                oaat_exp_id+=1
                oaat_exp_id_list.append("oaat_%i"%oaat_exp_id)
                oaat_exp_param_list.append(param)
                oaat_exp_param_value_list.append("x%0.2f"%value)
        else:
            value_min = np.float32(value_min)
            value_max = np.float32(value_max)
            step = (value_max-value_min)/(num_exp-1)
            for value in np.arange(value_min,value_max+1e-4,step):
                oaat_exp_id+=1
                oaat_exp_id_list.append("oaat_%i"%oaat_exp_id)
                oaat_exp_param_list.append(param)
                oaat_exp_param_value_list.append("%f"%value)

nroot Integer
4 0
hvt Continous
hvb Continous
rc Continous
cwpvt Continous
kc25 Continous
akc Continous
ko25 Continous
ako Continous
avcmx Continous
mp Continous
vcmx25 Continous
qe25 Continous
folnmx Continous
bb Continous
maxsmc Continous
satdk Continous
refdk_data Continous
refkdt_data Continous
ssi Continous
mfsno Continous
rsurf_snow Continous
tau0 Continous
snow_ret_fac Continous


In [8]:
df_exp_sum = pd.DataFrame(np.transpose([oaat_exp_param_list,oaat_exp_param_value_list]),
             index=oaat_exp_id_list,columns=['param','value'])

# Prep NoahMP Tables with correct parameters

In [9]:
noahmptable = 'noahmptable.tbl'

In [10]:
# make output directory
odir_oaat = "out/"
if not os.path.isdir(odir_oaat):
    os.mkdir(odir_oaat)

In [11]:
for row in df_exp_sum.iloc[3:4].iterrows():
    # read in the table
    dict_value,dict_format,dict_parameters_format_section,dict_parameters_section=read_noahmp_table(noahmptable)
    dict_parameters_section_exp = dict_parameters_section
    exp_id = row[0]
    param = row[1]['param']
    value = row[1]['value']
    value_type = df_sel['Continous or Integer'].loc[param]
    section_ = df.loc[param]['Section']
    section_list = section_.split('/')
    print(exp_id,param)
    for section in section_list:
        section_param = "%s:%s"%(section, param)
        default_values = dict_parameters_section[section][section_param]
        print(exp_id,param,value, "Default param", section, default_values)
        if 'x' in value:
            update_value = [float_nsf(x * np.float32(value.split('x')[1]),3) for x in default_values]
            dict_parameters_section_exp[section][section_param] = update_value
        elif value_type == "Integer":
            value = int(value)
            update_value = [value] * len(default_values)
            dict_parameters_section_exp[section][section_param] = update_value
        else:
            update_value = [float_nsf(np.float32(value),3)]*len(default_values)
            dict_parameters_section_exp[section][section_param] = update_value
        print(exp_id,param,value, "Updated param", section, update_value)
    print("------")
    ofilename = odir_oaat + 'noahmptable.%s.tbl'%exp_id
    write_noahmptbl(ofilename,
                    dict_value,
                    dict_format,
                    dict_parameters_format_section,
                    dict_parameters_section_exp)
    

oaat_4 nroot
oaat_4 nroot 3 Default param noahmp_modis_parameters [4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 1.0, 3.0, 1.0, 1.0, 0.0, 3.0, 3.0, 2.0]
oaat_4 nroot 3 Updated param noahmp_modis_parameters [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
------


In [ ]:
# from math import log10, floor
# def round_to_3(x):
#     return round(x, -int(floor(log10(abs(x))))-5)

In [ ]:
# round(1.23232e-10,3)
# f"{1.23232e-10:.3g}"

In [ ]:
# import os
# os.path.isdir

In [ ]:
# for exp_id in exp_df.columns.values:
#     print("start preparing noahmptbl for %s"%exp_id)
#     dict_parameters_section_exp = dict(dict_parameters_section)
#     for row in exp_df.loc[:,[col]].iterrows():
#         param = row[0] #['Parameter name']
#         section_ = df.loc[param]['Section']
#         section_list = section_.split('/')
#         value = row[1][exp_id]
#         for section in section_list:
#             section_param = "%s:%s"%(section, param)
#             default_values = dict_parameters_section[section][section_param]
#             if 'x' in value:
#                 update_value = [x * np.float32(value.split('x')[1]) for x in default_values]
#                 dict_parameters_section_exp[section][section_param] = update_value
#                 # print(param, update_value,value)
#                 # print(param, dict_parameters_section[section][section_param],value)
#     # if not os.path.isdir(odir):
#         # os.mkdir(odir)
#     ofile = open(odir + 'noahmptable.%s.tbl'%exp_id, 'w')
#     for key in dict_format.keys():
#         l_format = dict_format[key]
#         if l_format == '':
#             line = dict_value[key]
#         else:
#             vkey = dict_value[key]
#             section = vkey.split(':')[0]
#             param_value = dict_parameters_section_exp[section][vkey]
#             param_v_format = dict_parameters_format_section[section][vkey]
#             # print(param_value, param_v_format)
#             len_format = len(param_v_format)
#             len_value = len(param_value)
#             line = ''
#             for j,item in enumerate(param_v_format):
#                 if j == 0:
#                     line += item
#                     line += '='
#                 elif j == len_format-1:
#                     line += item
#                 else:
#                     if item == '':
#                         line += param_value[j-1]
#                     else:
#                         line += item%param_value[j-1]
#                     if j-1 < len_value-1:
#                         line += ','
#             # print([line])
#         ofile.write(line)
#     ofile.close()

In [ ]:
# exp_df

In [ ]:
# dict_parameters_section['noahmp_modis_parameters']['noahmp_modis_parameters:hvt']

In [ ]:
# for row in df.iterrows():
#     param = row[1]['Parameter name']
#     section_ = row[1]['Section']
#     if type(section_) == str:
#         if param in param_change_dic.keys():
#             section_list = section_.split('/')
#             for section in section_list:
#                 section_param = "%s:%s"%(section, param)
#                 default_values = row[1]['Default (MODIS)']
#                 # print(row[0],param,section, default_values)
#                 # print(row[0],dict_parameters_section[section][section_param])

In [ ]:
# dict_parameters_section['noahmp_modis_parameters']['noahmp_modis_parameters:nroot']

In [ ]:
# ofile = open('/glade/work/yifanc/proj/wpo2023/run/prod/CA-ARF/noahmptable.tbl.replicate1', 'w')

In [ ]:
# for key in dict_format.keys():
#     l_format = dict_format[key]
#     if l_format == '':
#         line = dict_value[key]
#     else:
#         vkey = dict_value[key]
#         section = vkey.split(':')[0]
#         param_value = dict_parameters_section[section][vkey]
#         param_v_format = dict_parameters_format_section[section][vkey]
#         # print(param_value, param_v_format)
#         len_format = len(param_v_format)
#         len_value = len(param_value)
#         line = ''
#         for j,item in enumerate(param_v_format):
#             if j == 0:
#                 line += item
#                 line += '='
#             elif j == len_format-1:
#                 line += item
#             else:
#                 if item == '':
#                     line += param_value[j-1]
#                 else:
#                     line += item%param_value[j-1]
#                 if j-1 < len_value-1:
#                     line += ','
#         # print([line])
#     ofile.write(line)
# ofile.close()

In [ ]:
# item

In [ ]:
# param_value[j-1]

# code development

In [ ]:
# line_split_equal = [' gdvic   ',
#                     '  0.050,  0.070,  0.130,  0.200,  0.170,  0.110,  0.260,  0.350,  0.260,   0.300,   0.380,   0.410,  0.500,  0.001,  0.010,  0.001,   0.001,  0.050,  0.020\n']
line_split_equal = ['lfpt_s3    ', '   0.36,    0.4,    0.4,    0.4,    0.4,\n']


In [ ]:
param_name = line_split_equal[0]
param_name_strip = param_name.strip()

# if not whether_exist_comment:
data_str,end_str = truncate_string(line_split_equal[1])
num_values_list = []
format_list = []
if ',' in data_str:
    values = re.split(',|\n', data_str)
    for value in values:
        # if has_numbers(value):
        value_num = value
        num = np.float(value_num)
        value_num_strip = value_num.strip()
        value_num_strip_format = string_data_format(value_num_strip)
        value_num_format = value_num.replace(value_num.strip(), value_num_strip_format)
        num_values_list.append(num)
        format_list.append(value_num_format)
        # else:
        #     num_values_list.append(value)
        #     format_list.append('')
else:
    value_num = data_str
    num = np.float(value_num)
    value_num_strip = value_num.strip()
    value_num_strip_format = string_data_format(value_num_strip)
    value_num_format = value_num.replace(value_num.strip(), value_num_strip_format)
    num_values_list.append(num)
    format_list.append(value_num_format)


In [ ]:
value_num = '   4.e-5   '
value = np.float(value_num)

In [ ]:
value_num_strip = value_num.strip()
value_num_strip_format = string_data_format(value_num_strip)

In [ ]:
value_num_format = value_num.replace(value_num.strip(), value_num_strip_format)

In [ ]:
value_num_format%(Decimal(value))

In [ ]:
from decimal import Decimal


In [ ]:
truncate_string(line_split_equal[1])

# code development

In [ ]:
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

def string_data_format(s):
    if '"' in s:
        format = "%s"
    elif 'e' in s:
        num_list = s.split('e')
        if '.' in s:
            num_digit = len(num_list[0].split('.')[0]) + len(num_list[0].split('.')[1]) - 1
            if num_digit <1:
                num_digit = 1
        else:
            num_digit = 1
        format = "%."+"%i"%(num_digit) + "e"
    elif 'E' in s:
        num_list = s.split('E')
        if '.' in s:
            num_digit = len(num_list[0].split('.')[1])
            if num_digit <1:
                num_digit = 1
        else:
            num_digit = 1
        format = "%."+"%i"%(num_digit) + "E"
    elif '.' in s:
        num_digit = len(s.split('.')[1])
        if num_digit == 0:
            num_digit = 1
        format = "%."+"%i"%(num_digit) + "f"
    else:
        format = "%i"
    return format

def truncate_string(input_str):
    """
    Truncate a string after the last occurrence of a number or a dot.

    Args:
        input_str (str): The input string to truncate.

    Returns:
        str: The truncated string.
    """
    last_index = max(input_str.rfind('.'), input_str.rfind('0'), input_str.rfind('1'), 
                     input_str.rfind('2'), input_str.rfind('3'), input_str.rfind('4'),
                     input_str.rfind('5'), input_str.rfind('6'), input_str.rfind('7'), 
                     input_str.rfind('8'), input_str.rfind('9'))

    if last_index == -1:
        return input_str,''
    else:
        return input_str[:last_index + 1],input_str[last_index+1:]

In [ ]:
def read_noahmp_table(fname):
    file1 = open(fname, 'r')
    # Using readlines()
    Lines = file1.readlines()
     
    count = 0
    # Strips the newline character
    dict_format = {}
    dict_value  = {}
    dict_parameters_section = {}
    dict_parameters_format_section = {}
    for line in Lines:
        count += 1
        # print("{}: {}".format(count, line))
        if len(line.strip())>0:
            # print(line.strip()[0])
            if line.strip()[0] == '&':
                # print(line)
                # print("section start for: %s"%line.strip()[1:])
                section_name = line.strip()[1:]
                dict_value.update({count:line})
                dict_format.update({count:''})
                dict_parameters = {}
                dict_parameters_format = {}
            elif line.strip()[0] == '!':
                dict_value.update({count:line})
                dict_format.update({count:''})
            elif line.strip()[0].isalpha():
                param_value_list = []
                whether_exist_comment = False
                if '!' in line:
                    line_split_exc = line.split('!')
                    # print(line_split_exc)
                    param_value_list.append("!"+line_split_exc[-1])
                    # print(param_value_list)
                    line = line_split_exc[0]
                    whether_exist_comment = True
                line_split_equal = line.split('=')
                # print(line_split_equal)
                param_name = line_split_equal[0]
                param_name_strip = param_name.strip()
                
                # if comment exists: we need to update the last string
                data_str,end_str = truncate_string(line_split_equal[1])
                if whether_exist_comment:
                    post_str = end_str + param_value_list[0]
                    param_value_list = []
                else:
                    post_str = end_str
                param_value_list.append(post_str)
    
                # start extract variables from 
                
                num_values_list = []
                format_list = []
                if ',' in data_str:
                    values = re.split(',|\n', data_str)
                    for value in values:
                        # if has_numbers(value):
                        value_num = value
                        num = np.float(value_num)
                        value_num_strip = value_num.strip()
                        value_num_strip_format = string_data_format(value_num_strip)
                        value_num_format = value_num.replace(value_num.strip(), value_num_strip_format)
                        num_values_list.append(num)
                        format_list.append(value_num_format)
                        # else:
                        #     num_values_list.append(value)
                        #     format_list.append('')
                else:
                    value_num = data_str
                    if '"' in value_num:
                        num_values_list.append(value_num)
                        format_list.append('')
                    else:
                        num = np.float(value_num)
                        value_num_strip = value_num.strip()
                        value_num_strip_format = string_data_format(value_num_strip)
                        value_num_format = value_num.replace(value_num.strip(), value_num_strip_format)
                        num_values_list.append(num)
                        format_list.append(value_num_format)
                # print(num_values_list,format_list)
                dict_parameters.update({"%s:%s"%(section_name,param_name_strip):num_values_list})
                dict_parameters_format.update({"%s:%s"%(section_name,param_name_strip):[param_name]+format_list+[post_str]})
                dict_value.update({count:"%s:%s"%(section_name,param_name_strip)})
                dict_format.update({count:'customized'})
            elif line.strip()[0] == '/':
                dict_parameters_section.update({section_name:dict_parameters})
                dict_parameters_format_section.update({section_name:dict_parameters_format})
                dict_value.update({count:line})
                dict_format.update({count:''})
            else:
                dict_value.update({count:line})
                dict_format.update({count:''})
        else:
            dict_value.update({count:line})
            dict_format.update({count:''})
    return dict_value,dict_format,dict_parameters_format_section,dict_parameters_section

In [ ]:
def write_noahmptbl(ofilename,dict_value,dict_format,dict_parameters_format_section,dict_parameters_section):
    ofile = open(ofilename, 'w')
    for key in dict_format.keys():
        l_format = dict_format[key]
        if l_format == '':
            line = dict_value[key]
        else:
            vkey = dict_value[key]
            section = vkey.split(':')[0]
            param_value = dict_parameters_section[section][vkey]
            param_v_format = dict_parameters_format_section[section][vkey]
            # print(param_value, param_v_format)
            len_format = len(param_v_format)
            len_value = len(param_value)
            line = ''
            for j,item in enumerate(param_v_format):
                if j == 0:
                    line += item
                    line += '='
                elif j == len_format-1:
                    line += item
                else:
                    if item == '':
                        line += param_value[j-1]
                    else:
                        line += item%param_value[j-1]
                    if j-1 < len_value-1:
                        line += ','
            # print([line])
        ofile.write(line)
    ofile.close()

In [ ]:
def float_nsf(num, n=1):
    """Converts a float to a string with n significant figures."""
    numstr = ("{0:.%ie}" % (n - 1)).format(num)
    return float(numstr)